In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [ ]:
if IN_COLAB:
    !git clone https://github.com/LeonLaumeyer/mai_project1_optimization.git

In [ ]:
if IN_COLAB:
    !pip3 install -r mai_project1_optimization/requirements.txt
    !pip3 install optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random
import multiprocessing

if(IN_COLAB):
    from mai_project1_optimization.modules.dataset import IntelImageClassificationDataset
    from mai_project1_optimization.modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from mai_project1_optimization.modules.trainer import Trainer
    from mai_project1_optimization.modules.optuna_optimizer import OptunaTuner
    from mai_project1_optimization.modules.optuna_monashara import run_optuna
    from mai_project1_optimization.modules.BufferDataset import ShuffleBufferDataset
else:
    from modules.dataset import IntelImageClassificationDataset
    from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from modules.trainer import Trainer
    from modules.optuna_optimizer import OptunaTuner
    from modules.optuna_monashara import run_optuna
    from modules.BufferDataset import ShuffleBufferDataset

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

# Removed support for Tensor Units
# torch.backends.cudnn.allow_tf32 = True
# torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

cpu


https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [ ]:
# labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
# fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
# fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset)

In [ ]:
choice = 1 # 1,2,3
freezeLayer = False
pretrained_Weights = False
prune_model = False
USE_OPTUNA = False
OPTUNA_MO = False
BUFFER_MO = False
MULTIPROCESSING_B = False
SHUFFLE_BUFFER_SIZE = 7500

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# 80% train, 20% validation for training Optuna
train_size = int(0.8 * len(dataset.train_dataset))
val_size = len(dataset.train_dataset) - train_size
train_subset, val_subset = random_split(dataset.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(1))

def build_model():

  # SqueezeNet 1.1
  if choice == 1:
      if(pretrained_Weights):
        model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
      else:
        model = models.squeezenet1_1()

      num_features = model.classifier[1].in_channels
      kernel_size = model.classifier[1].kernel_size
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)


  # MobileNetV2
  elif choice == 2:
      if(pretrained_Weights):
        model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
      else:
        model = models.mobilenet_v2()

      num_features = model.classifier[1].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Linear(num_features, 6)

  # MobileNetV3 Small
  elif choice == 3:
      if(pretrained_Weights):
        model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
      else:
        model = models.mobilenet_v3_small()
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # MobileNetV3 Large
  elif choice == 4:
      if(pretrained_Weights):
        model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
      else:
        model = models.mobilenet_v3_large()
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # VisionTransformer Base 16
  elif choice == 5:
      if(pretrained_Weights):
        model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)
      else:
        model = models.vit_b_16()
      num_features = model.heads[0].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.heads[0] = nn.Linear(num_features, 6)

  if prune_model:
    model = apply_pruning(model, amount=0.3)

  return model

if USE_OPTUNA:
    tuner = OptunaTuner(
        model_fn=build_model,
        train_dataset=train_subset,
        val_dataset=val_subset,
        device=DEVICE
    )
    study = tuner.run(n_trials=10, seed=1)
    print("Best trial parameters:", study.best_trial.params)

    best_params = study.best_trial.params
    model = build_model()
    dataloader = DataLoader(train_subset, batch_size=best_params["batch_size"], shuffle=True)
    trainer = Trainer(model=model, lr=best_params["lr"], device=DEVICE)
    epochs = best_params["epochs"]

elif OPTUNA_MO:

    model = build_model()

    best_params, best_model_state, study = run_optuna(
        model=model,
        train_subset=train_subset,
        val_subset=val_subset,
        TrainerClass=Trainer,
        n_trials=12,
        seed=1
    )

    print("▶ Per-epoch validation accuracy (best trial):")
    best_trial = study.best_trial
    for epoch, acc in sorted(best_trial.intermediate_values.items()):
        print(f"   Epoch {epoch:2d}: {acc * 100:.2f}%")

    print(f"\n▶ Best hyperparameters: {best_params}")
    print(f"▶ Best overall accuracy: {study.best_value * 100:.2f}%")

    model.load_state_dict(best_model_state)

    train_ds = ShuffleBufferDataset(dataset.train_dataset, buffer_size=SHUFFLE_BUFFER_SIZE) \
               if BUFFER_MO else dataset.train_dataset
    dataloader = DataLoader(train_ds, batch_size=best_params["BS_SUGGEST"], shuffle=not BUFFER_MO)
    dataloader = DataLoader(dataset.train_dataset, batch_size=best_params["BS_SUGGEST"], shuffle=True)
    trainer = Trainer(model=model,lr=best_params["LR_SUGGEST"],device=DEVICE)
    epochs = best_params["EPOCHS"]

    ''' BS_SUGGEST': 32, 'LR_SUGGEST': 1.6141462555811457e-05, 'EPOCHS': 25 suggested from the OPTUNA
        and achieve the accuracy of 91.35 % on Testdata.'''

else:
    model = build_model()
    train_ds = ShuffleBufferDataset(dataset.train_dataset, buffer_size=SHUFFLE_BUFFER_SIZE) \
               if BUFFER_MO else dataset.train_dataset
    dataloader = DataLoader(train_ds, batch_size=32, shuffle=not BUFFER_MO)
    #dataloader = DataLoader(dataset.train_dataset, batch_size=32, shuffle=True)
    trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)
    epochs = 25

############## Multiprocessing - Felix Krah (start)###########
if MULTIPROCESSING_B: # Change of some dataloader attributes - kind of depends on values like batch size though
    workers = multiprocessing.cpu_count()
    prefac = 4
    if not IN_COLAB:
        workers = workers // 2
    else:
        prefac = 2
    dataloader = DataLoader(
    dataset = dataloader.dataset,
    batch_size = dataloader.batch_size,
    sampler = dataloader.sampler,
    num_workers = workers, # Change
    pin_memory = True, # Change
    drop_last = dataloader.drop_last,
    timeout = dataloader.timeout,
    generator = dataloader.generator,
    prefetch_factor = prefac, # Change
    persistent_workers = True, # Change
    in_order = dataloader.in_order
    )
############## Multiprocessing - Felix Krah (end)###########

In [ ]:
# model.load_state_dict(torch.load(f"checkpoints/.pt"))
trainer.train(dataloader, epochs=epochs, silent=False)

In [ ]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

0.9018919467926025

In [ ]:
# torch.save(model.state_dict(), f"checkpoints/{model.__class__.__name__}.pt")

## Initial Results for Model Selection

| model | accuracy | size |
| --- | --- | --- |
| ResNet18 | 0.87 | 44.7 MB |
| ResNet34 | 0.88 | 83.3 MB |
| MobileNet V2 | 0.91 | 13.6 MB |
| MobileNet V3 small | 0.90 | 9.8 MB |
| VGG19 | 0.83 | 548.1 MB |
| SqueezeNet 1.0 | 0.89 | 4.8 MB |
| DenseNet | 0.90 | 30.8 MB |
| EfficientNet B0 | 0.92 | 20.5 MB |
| ViT-b/16 | 0.73 | 330.3 MB |